# 网络中的网络（NiN）
LeNet、AlexNet和VGG都有一个共同的设计模式：通过一系列的卷积层与汇聚层来提取空间结构特征；然后通过全连接层对特征的表征进行处理。AlexNet和VGG对LeNet的改进主要在于如何扩大和加深这两个模块。

如果在这个过程的早期就加入全连接层呢？那么可能会完全放弃表征的空间结构。但*网络中的网络*（*NiN*）提供了一个非常简单的解决方案：在每个像素的通道上分别使用多层感知机 :cite:`Lin.Chen.Yan.2013`

## NiN块
回想一下，卷积层的输入和输出由四维张量组成，张量的每个轴分别对应样本、通道、高度和宽度。另外，全连接层的输入和输出通常是分别对应于样本和特征的二维张量。

NiN的想法是在每个像素位置（针对每个高度和宽度）应用一个全连接层。如果我们将权重连接到每个空间位置，我们可以将其视为$1\times 1$卷积层（如 :numref:`sec_channels`中所述），或作为在每个像素位置上独立作用的全连接层。从另一个角度看，即将空间维度中的每个像素视为单个样本，将通道维度视为不同特征（feature）。

 :numref:`fig_nin`说明了VGG和NiN及它们的块之间主要架构差异。
NiN块以一个普通卷积层开始，后面是两个$1 \times 1$的卷积层。这两个$1 \times 1$卷积层充当带有ReLU激活函数的逐像素全连接层。第一层的卷积窗口形状通常由用户设置。随后的卷积窗口形状固定为$1 \times 1$。

![对比 VGG 和 NiN 及它们的块之间主要架构差异。](./nin.svg)
:width:`600px`
:label:`fig_nin`


In [1]:
import torch
import torch.nn as nn 

In [2]:
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 
                  kernel_size=kernel_size, stride=strides, padding=padding), 
        nn.ReLU(), 
        
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU(), 
        
        nn.Conv2d(out_channels, out_channels, kernel_size=1), 
        nn.ReLU(), 
        )

## NiN模型
最初的NiN网络是在AlexNet后不久提出的，显然从中得到了一些启示。
NiN使用窗口形状为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，输出通道数量与AlexNet中的相同。
每个NiN块后有一个最大汇聚层，汇聚窗口形状为$3\times 3$，步幅为2。

NiN和AlexNet之间的一个显著区别是NiN完全取消了全连接层。相反，NiN使用一个NiN块，其输出通道数等于标签类别的数量。最后放一个*全局平均汇聚层*（global average pooling layer），生成一个对数几率	（logits）。NiN设计的一个优点是，它显著减少了模型所需参数的数量。然而，在实践中，这种设计有时会增加训练模型的时间。

In [3]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    nin_block(96, 256, kernel_size=5, strides=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用全局平均池化层，有多少个类别，就有多少个输入通道
    nn.AdaptiveAvgPool2d((1, 1)),
    
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten()
    )

我们创建一个数据样本来[查看每个块的输出形状]。

In [4]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 24, 24])
MaxPool2d output shape:	 torch.Size([1, 256, 11, 11])
Sequential output shape:	 torch.Size([1, 384, 11, 11])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
MaxPool2d output shape:	 torch.Size([1, 10, 2, 2])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


## 小结
* NiN使用由一个卷积层和多个$1\times 1$卷积层组成的块。
* NiN去除了容易造成过拟合的全连接层，将它们替换为全局平均汇聚层
* 移除全连接层可减少过拟合，同时显著减少NiN的参数。
* NiN的设计影响了许多后续卷积神经网络的设计。